# Task 1

The first task tests your Python skills. You need to develop a simple game consisting of a rectangular grid (of size height x width ) where each cell has a random integer value between 0 and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right corner of the grid as fast as possible.

In [1]:
import numpy as np
import random
from abc import ABC, abstractmethod

In [2]:
class Game():
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.maze = (np.random.rand(self.height, self.width) * 10).astype('int')
        self.start = self.maze[0][0]
        self.end = self.maze[self.height - 1][self.width - 1]
        self.game_complete = False
        # Pointer
        self.pointer_x = 0
        self.pointer_y = 0
        self.pointer_val = self.maze[0][0]
        self.pointer = f"Currently at [{0}, {0}], of cost {self.start}"
        # Records
        self.steps_taken = 0
        self.time_spent = 0
    

    #Move the pointer from its current position, to the position passed in as parameters (x, y)
    def move_pointer(self, x_pos, y_pos):
        # Check if x_pos and y_pos are legal (step size of 1 unit, that does not exceed maze boundary)
        if(abs(x_pos - self.pointer_x) > 1 or (x_pos >= self.width or x_pos < 0)):
            print("X Step too large")
            return
        if(abs(y_pos - self.pointer_y) > 1 or (y_pos >= self.height or y_pos < 0)):
            print("Y Step too large")
            return
        # Check that step is not diagonal (i.e. it is not the case that both x and y positions have changed)
        if((x_pos != self.pointer_x) and (y_pos != self.pointer_y)):
            print("Cannot move diagonally!")
            return
        # If all is legal, make the step
        # Update records
        self.steps_taken += 1
        self.time_spent += self.pointer_val
        # Move pointer
        self.pointer_x = x_pos
        self.pointer_y = y_pos
        self.pointer_val = self.maze[y_pos][x_pos]
        self.pointer = f"Currently at [{x_pos}, {y_pos}], of cost {self.pointer_val}"
        print(f"{self.pointer}")
        # Check if after the move, the agent has won the game.
        if(self.pointer_x == (self.width - 1) and self.pointer_y == (self.height - 1)):
            self.game_complete = True
            print("Won the game! 🥳")
    

    #Reset the game
    def reset(self):
        print("Resetting game...")
        self.game_complete = False
        self.pointer_x = 0
        self.pointer_y = 0
        self.steps_taken = 0
        self.time_spent = 0
        self.pointer = f"Currently at [{0}, {0}], of cost {self.start}"
        print(self.pointer)
    
    def get_cost(self, x_pos, y_pos):
        return self.maze[y_pos][x_pos]


    #Given a point on the grid (x, y), return an array of neighboring coordinates [[x1, y1], [x2, y2], [x3, y3]...etc.]
    def get_neighbors(self, x_pos, y_pos):
        x_plus = x_pos + 1 if x_pos < (self.width - 1) else None
        x_minus = x_pos - 1 if x_pos > 0 else None
        y_plus = y_pos + 1 if y_pos < (self.height - 1) else None
        y_minus = y_pos - 1 if y_pos > 0 else None
        
        above = [x_pos, y_minus]
        below = [x_pos, y_plus]
        left = [x_minus, y_pos]
        right = [x_plus, y_pos]

        neighbors = [above, below, left, right]

        result = [ n for n in neighbors if (n[0] != None and n[1] != None)]

        return result




In [3]:
#Test game
game = Game(width=3, height=3)
print(f"{game.maze}\n")

print("After a move to x=1, y=0")
game.move_pointer(x_pos=1, y_pos=0)

[[3 7 6]
 [3 7 1]
 [5 6 9]]

After a move to x=1, y=0
Currently at [1, 0], of cost 7


# Agents

After implementing the game, we are tasked with developing agents to play it. All agents will conform to the Agent abstract base class. They must all take a game (type Game) into their constructor, and implement a method play_game() which plays the game that the agent was initialized with, as well as the score() method, which prints the agent's performance in completing the game.

In [4]:
class Agent(ABC):
    @abstractmethod
    def __init__(self, game: Game):
        self.game = game
        self.steps_taken = 0
        self.time_spent = 0
        
    @abstractmethod
    def play_game(self):
        pass
    
    @abstractmethod
    def score(self):
        pass

## Random Agent

The first agent we need to develop is a "basline agent". The baseline agent needs to be superior to a random agent in reaching the end of the game. We implement a random agent to test their performance for comparison's sake.

In [ ]:
class Random_Agent(Agent):
    def __init__(self, game: Game):
        self.game = game
        self.steps_taken = 0
        self.time_spent = 0
    
    def play_game(self):
        self.game.reset()
        # While the game is not complete...
        while(not self.game.game_complete):
            # Determine if we're moving along the y axis (if not, the step is along the x axis)
            move_y = random.choice((0, 1))
            # Determine if our direction is negative or positive (e.g. backwards along the y axis vs forwards along the y axis)
            direction = random.choice((-1, 1))

            # If the move is along the x axis, ensure the move does not exceed the boundaries of the maze (in that case, go back to start of loop)
            # Then, move in the random direction alongside the x axis
            if(move_y == 0):
                new_x = self.game.pointer_x + direction
                new_y = self.game.pointer_y
                if(new_x >= self.game.width or new_x < 0):
                    continue
                # print("Moving x")
                self.game.move_pointer(x_pos=new_x, y_pos=new_y)
            #Do the same if it were determined that the move should be along the y axis...
            else:
                new_x = self.game.pointer_x
                new_y = self.game.pointer_y + direction
                if(new_y >= self.game.height or new_y < 0):
                    continue
                # print("Moving y")
                self.game.move_pointer(x_pos=new_x, y_pos=new_y)
        
        self.steps_taken = self.game.steps_taken
        self.time_spent = self.game.time_spent
        return

    def score(self):
        print(f"\nTime spent: {self.time_spent}, Steps taken: {self.steps_taken}")


In [ ]:
# game2 = Game(width=6, height=4)
# print(f"{game2.maze}\n")
random_agent = Random_Agent(game)
random_agent.play_game()
random_agent.score()

# Baseline Agent

Our baseline agent will move diagonally towards the bottom right. Once it reaches an edge, it then moves (either downwards or across to the left, depending on maze dimensions), until it reaches the end point

In [ ]:
class Baseline_Agent(Agent):
    def __init__(self, game: Game):
        self.game = game
        self.steps_taken = 0
        self.time_spent = 0
    

    def play_game(self):
        self.game.reset()
        while(not self.game.game_complete):
            # If we reached the bottom edge, move across to the right
            if (self.game.pointer_y == self.game.height - 1):
                new_x = self.game.pointer_x + 1
                new_y = self.game.pointer_y
                self.game.move_pointer(x_pos=(new_x), y_pos=(new_y))

            # Else if we reached the right edge, move downwards
            elif (self.game.pointer_x == self.game.width - 1):
                new_x = self.game.pointer_x
                new_y = self.game.pointer_y + 1
                self.game.move_pointer(x_pos=(new_x), y_pos=(new_y))

            # Else, move diagonally, by going through two steps
            else:
                new_x_1 = self.game.pointer_x + 1
                new_y_1 = self.game.pointer_y
                self.game.move_pointer(x_pos=(new_x_1), y_pos=(new_y_1))
                new_x_2 = self.game.pointer_x
                new_y_2 = self.game.pointer_y + 1
                self.game.move_pointer(x_pos=(new_x_2), y_pos=(new_y_2))

        self.steps_taken = self.game.steps_taken
        self.time_spent = self.game.time_spent
        return
    
    def score(self):
        print(f"\nTime spent: {self.time_spent}, Steps taken: {self.steps_taken}")
            

In [ ]:
# game3 = Game(width=20, height=4)
# print(f"{game3.maze}\n")
baseline_agent = Baseline_Agent(game)
baseline_agent.play_game()
baseline_agent.score()

# Dijkstra's Algorithm

We build an agent that plays the game using Dijkstra's algorithm.    

In [ ]:
class Dijkstra_Agent(Agent):
    def __init__(self, game: Game):
        self.game = game
        self.steps_taken = 0
        self.time_spent = 0

        self.number_of_nodes = self.game.width * self.game.height
        self.current_node = f"{0},{0}"
        self.visted_nodes = []
        self.path = []

        #Initialize dictionary info, whose schema looks like:
        #      {“node”: {previous: “node”, cost: 0}...}
        # Where "node" is a string representation of the coordinates of a cell on the grid... e.g. "0,0" for [0, 0]
        self.info = {}
        # Fill info with values given the game's dimensions
        for w in range(self.game.width):
            for h in range(self.game.height):
                if(w == 0 and h == 0 ):
                    self.info[f"{w},{h}"] = {"prev": None, "cost": 0}
                else:
                    self.info[f"{w},{h}"] = {"prev": None, "cost": float("inf")}
    

    def play_game(self):
        self.game.reset()
        # Fills in self.info "cost" and "prev" values for each node
        self.explore()
        # Uses self.info to build up the quickest path to endpoint
        self.build_path()
        # Play the game by following the path
        for n in self.path:
            move_cor = self.string_to_coor(n)
            self.game.move_pointer(x_pos=move_cor[0], y_pos=move_cor[1])
        
        self.steps_taken = self.game.steps_taken
        self.time_spent = self.game.time_spent
        

    def build_path(self):
        # Initialize "node" to equal the end point
        node = f"{self.game.width - 1},{self.game.height - 1}"
        # Working backwards, check the value for "prev" in self.info, corresponding to the node defined above
        # This will give us the quickest path going backwards, starting from the end, and reaching the start
        # Insert every encountered node on this path going backwards, to the front of self.path. 
        # The end result is a list of corrodinates for the quickest path, from start to end
        while not node == f"{0},{0}":
            self.path.insert(0, node)
            node = self.info[node]["prev"]


    def explore(self):
        #Base case - if we've visted all nodes, we're done
        if len(self.visted_nodes) >= self.number_of_nodes:
            return

        #Convert self.current_node into numerical format using self.string_to_coor.. e.g. "0,0" -> [0, 0]
        current_node_numb = self.string_to_coor(self.current_node)
        #Get all the neighbors of self.current_node (helper function that outputs all neighbors is in Game class)
        current_neighbors = self.game.get_neighbors(current_node_numb[0], current_node_numb[1])

        #For each neighbor that has not been previously visited (i.e. is not in self.visited_nodes) do the following:
        # - Initialize a variable called "current_cost". 
        #   Its value is the current_node's cost as specified in self.info (total cost from start to current_node), plus the cost of traveling to this neighbor from current_node
        # - If value for the variable "current_cost" is less than the neighbor's cost as specified in self.info (total cost from start to that neighbor), 
        #   set info["neighbor"]["cost"] to current_cost, and info["neighbor"]["prev"] to current_node
        for n in current_neighbors:
            if not n in self.visted_nodes:
                current_cost = self.info[self.current_node]["cost"] + self.game.get_cost(x_pos=n[0], y_pos=n[1])
                if current_cost <  self.info[f"{n[0]},{n[1]}"]["cost"]:
                    self.info[f"{n[0]},{n[1]}"]["cost"] = current_cost
                    self.info[f"{n[0]},{n[1]}"]["prev"] = self.current_node
        
        # Tick current_node off by adding it to self.visited_nodes
        self.visted_nodes.append(self.current_node)

        # Set current_node to equal to the non visited node with the lowest cost as specified in info (total cost from start to that node)
        self.update_current_node()

        #Recursivley call explore again
        self.explore()


    def update_current_node(self):
        if len(self.visted_nodes) >= self.number_of_nodes:
            return

        least_cost = float("inf")
        least_cost_node = ""

        for n in self.info:
            if (not n in self.visted_nodes and self.info[n]["cost"] < least_cost):
                least_cost = self.info[n]["cost"]
                least_cost_node = n
        
        self.current_node = least_cost_node


    #Takes a string s, and returns an array of the corrdinates... e.g. "0,0" -> [0, 0]
    def string_to_coor(self, input):
        split_string = input.split(",")
        for i in range(len(split_string)):
            split_string[i] = int(split_string[i])
        return split_string

    
    def score(self):
        print(f"\nTime spent: {self.time_spent}, Steps taken: {self.steps_taken}")

            



In [ ]:
# game4 = Game(width=6, height=4)
# print(f"{game4.maze}\n")
d_agent = Dijkstra_Agent(game)
d_agent.play_game()
d_agent.score()

# Ant Colony Optimization Algorithm
We build an agent that plays the game using any colony optimization

In [69]:
class Ant_Agent(Agent):
    def __init__(self, game: Game, ants=10, batches=10, eva_factor=0.5, q_const=1):
        self.game = game
        self.steps_taken = 0
        self.time_spent = 0

        self.ants= ants
        self.batches = batches
        self.end_point = [(self.game.width - 1), (self.game.height - 1)]
        self.visited_nodes = []
        self.eva_factor = eva_factor
        self.q_const = q_const

        # #Force minimum size of ants
        # self.total_nodes = self.game.width * self.game.height
        # if self.ants < self.total_nodes:
        #     self.ants = self.total_nodes

        # #Force minimum size of batches
        # self.min_batches = self.ants * 5
        # if self.batches < self.min_batches:
        #     self.batches = self.min_batches

        self.ant_info = {}
        for ant in range(self.ants):
            self.ant_info[ant] = [[0,0]]
        
        self.node_p_levels = {}
        for w in range(self.game.width):
            for h in range(self.game.height):
                if(w == 0 and h == 0 ):
                    self.node_p_levels[f"{w},{h}"] = 0
                else:
                    self.node_p_levels[f"{w},{h}"] = 0
    
    def play_game(self):
        self.game.reset()
        self.explore()
        print("\nBegin playing...")
        while (not self.game.game_complete):
            self.move()

    def move(self):
        loc = [self.game.pointer_x, self.game.pointer_y]

        neighbors = self.game.get_neighbors(x_pos=loc[0], y_pos=loc[1])
        neighbors = [ n for n in neighbors if not n in self.visited_nodes]

        max_p_level = self.node_p_levels[f"{neighbors[0][0]},{neighbors[0][1]}"]
        max_p_node = neighbors[0]

        for n in neighbors:
            if self.node_p_levels[f"{n[0]},{n[1]}"] > max_p_level:
                max_p_level = self.node_p_levels[f"{n[0]},{n[1]}"]
                max_p_node = n
        
        if self.end_point in neighbors:
            max_p_node = self.end_point

        self.visited_nodes.append(max_p_node)
        self.game.move_pointer(x_pos=max_p_node[0], y_pos=max_p_node[1])



    def explore(self):
        for b in range(self.batches):
            # Let each ant come to a solution
            print("Releasing the ants...")
            for ant in range(self.ants):
                print(f"Ant {ant} in batch {b} begins its journey")
                current_ant_loc = self.ant_info[ant][len(self.ant_info[ant]) - 1]
                while(not current_ant_loc == self.end_point):
                    chosen_neighbor = self.select_neighbor(loc=current_ant_loc)
                    self.ant_info[ant].append(chosen_neighbor)
                    current_ant_loc = chosen_neighbor
            # Update phermone levels
            print("Updating Phermones...")
            for ant in range(self.ants):
                list_of_costs = []
                for n in self.ant_info[ant]:
                    cost = self.game.get_cost(x_pos=n[0], y_pos=n[1])
                    if(not cost == 0):
                        list_of_costs.append(self.q_const / cost)
                    else:
                        list_of_costs.append(0)
                # print(f"list of costs {list_of_costs}")
                for n in self.ant_info[ant]:
                    current_p_level = self.node_p_levels[f"{n[0]},{n[1]}"]
                    # print(f"current_p_level is {current_p_level}")
                    self.node_p_levels[f"{n[0]},{n[1]}"] = (1 - self.eva_factor) * current_p_level + sum(list_of_costs)
    

    #A function that takes a location on the grid, and selects an approrpaite neighbor for the ant to travel to
    def select_neighbor(self, loc):
        # print(f"Selecting neigh to location {loc}")
        neighbors = self.game.get_neighbors(x_pos=loc[0], y_pos=loc[1])
        # print(f"neighbrs are {neighbors}")

        #If the end_point is a neighbor, select it
        if(self.end_point in neighbors):
            # print("Endpoint in neighbors")
            return self.end_point

        #For each n in neighbors, build a probabilities array with the probability to take that neighbor
        #Probability to select neighbor is node_p_levels[n] / total_neighbor_p_level

        #First calcuate total_neighbor_p_level
        total_neighbor_p_level = 0
        for n in neighbors:
            total_neighbor_p_level += self.node_p_levels[f"{n[0]},{n[1]}"]
        
        #If all neighbors have p_level of 0, chose a random neighbor to go to
        if total_neighbor_p_level == 0:
            return random.choice(neighbors)
        
        #Else choose a neighbor based on how high the p_levels are relative to other neighbors
        #Build an array of probabilities (we need to build a stringified version of neighbors to use np.random.choice as well)
        #Reference: https://docs.scipy.org/doc//numpy-1.10.4/reference/generated/numpy.random.choice.html
        probabilities = []
        neighbors_s = []
        for n in neighbors:
           probabilities.append(self.node_p_levels[f"{n[0]},{n[1]}"] / total_neighbor_p_level)
           neighbors_s.append(f"{n[0]},{n[1]}")
        
        return self.string_to_coor(np.random.choice(neighbors_s, p=probabilities))


        
    #Takes a string s, and returns an array of the corrdinates... e.g. "0,0" -> [0, 0]
    def string_to_coor(self, input):
        split_string = input.split(",")
        for i in range(len(split_string)):
            split_string[i] = int(split_string[i])
        return split_string
    
    def score(self):
        pass


In [76]:
game64 = Game(width=5, height=4)
print(f"{game64.maze}\n")
ant_agent = Ant_Agent(game=game64, ants=30, batches=10, eva_factor=0.5, q_const=1)
ant_agent.play_game()

[[1 7 0 4 6]
 [3 1 7 1 4]
 [1 5 4 2 1]
 [3 9 2 7 3]]

Resetting game...
Currently at [0, 0], of cost 1
Releasing the ants...
Ant 0 in batch 0 begins its journey
Ant 1 in batch 0 begins its journey
Ant 2 in batch 0 begins its journey
Ant 3 in batch 0 begins its journey
Ant 4 in batch 0 begins its journey
Ant 5 in batch 0 begins its journey
Ant 6 in batch 0 begins its journey
Ant 7 in batch 0 begins its journey
Ant 8 in batch 0 begins its journey
Ant 9 in batch 0 begins its journey
Ant 10 in batch 0 begins its journey
Ant 11 in batch 0 begins its journey
Ant 12 in batch 0 begins its journey
Ant 13 in batch 0 begins its journey
Ant 14 in batch 0 begins its journey
Ant 15 in batch 0 begins its journey
Ant 16 in batch 0 begins its journey
Ant 17 in batch 0 begins its journey
Ant 18 in batch 0 begins its journey
Ant 19 in batch 0 begins its journey
Ant 20 in batch 0 begins its journey
Ant 21 in batch 0 begins its journey
Ant 22 in batch 0 begins its journey
Ant 23 in batch 0 begins its journ

In [71]:
print(ant_agent.node_p_levels)

{'0,0': 54.52339318103862, '0,1': 65.41175020882463, '0,2': 66.6483686148286, '0,3': 66.86916044385512, '0,4': 66.14645669291338, '1,0': 67.6012164216688, '1,1': 67.59985255919314, '1,2': 67.563674781302, '1,3': 67.53684455111028, '1,4': 65.02804077642786, '2,0': 67.54788674336795, '2,1': 67.46197237715889, '2,2': 67.300235749102, '2,3': 65.26354176658151, '2,4': 53.42607704774873, '3,0': 66.68449937368923, '3,1': 67.584260549726, '3,2': 66.83077261042105, '3,3': 40.75052898144505, '3,4': 51.53783922171013, '4,0': 60.77520579007858, '4,1': 67.3205666173042, '4,2': 67.48646034216577, '4,3': 44.50061443932408, '4,4': 45.00060202640847}
